In [21]:
import time
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys

sys.path.insert(0,"../")

from app.mlcode.predict_price_movements import CoinPricePredictor
from app.mlcode.determine_trading_state import DetermineTradingState
from app.mlcode.utils import read_in_data, read_in_yaml, running_on_aws, setup_logging, update_yaml_config

%pylab inline

import logging

from tqdm.notebook import tqdm
tqdm.pandas()

logger = logging.getLogger(__name__)

logger.setLevel(logging.CRITICAL)

import warnings; warnings.simplefilter('ignore')


Populating the interactive namespace from numpy and matplotlib


/Users/jonathanhilgart/Library/Caches/pypoetry/virtualenvs/go-trader-aTWd7ZB6-py3.8/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [22]:

logging.disable(sys.maxsize)

## Setup data
- All of these configs are stored in the notebooks folder

In [23]:
def read_in_config_files(base_path: str = "training_configs/", coin: str = "btc"):

    coin_to_predict = coin +"_"

    constants = read_in_yaml( base_path + "constants.yml", running_on_aws())

    trading_state_filename = base_path + coin_to_predict + "trading_state_config.yml"
    trading_constants = read_in_yaml(trading_state_filename, running_on_aws())

    won_lost_amount_filename = base_path + coin_to_predict + "won_and_lost_config.yml"
    won_and_lost_amount_constants = read_in_yaml(won_lost_amount_filename, running_on_aws())


    actions_to_take_filename =  base_path + coin_to_predict + "actions_to_take.yml"
    actions_to_take_constants = read_in_yaml(actions_to_take_filename, running_on_aws())

    ml_constants = read_in_yaml(base_path +constants["ml_config_filename"].split("/")[1], running_on_aws())
    
    all_predictions_filename = base_path + coin_to_predict + "all_predictions.csv"
    
    return coin, constants , trading_constants, won_and_lost_amount_constants, actions_to_take_constants, ml_constants, all_predictions_filename,  trading_state_filename, actions_to_take_filename, won_lost_amount_filename
                                                    
                                                    
                                                    

In [24]:
coin, constants , trading_constants, won_and_lost_amount_constants, actions_to_take_constants, ml_constants, all_predictions_filename,  trading_state_filename, actions_to_take_filename, won_lost_amount_filename = read_in_config_files(coin="btc")

In [25]:
all_predictions_filename

'training_configs/btc_all_predictions.csv'

- make sure we've reset our configs

In [26]:
trading_constants # ensure all zeros
assert trading_constants['buy_entry_price'] == 0
assert trading_constants['position_entry_date'] == None
trading_constants

{'buy_entry_price': 0.0,
 'buy_has_crossed_mean': 0.0,
 'mode': 'no_position',
 'position_entry_date': None,
 'short_entry_price': 0.0,
 'short_has_crossed_mean': 0.0,
 'stop_loss_pct': 0.1,
 'stop_loss_price': 0.0}

In [27]:
assert won_and_lost_amount_constants['dollar_amount_buy_lost'] == 0
assert won_and_lost_amount_constants['dollar_amount_buy_won'] == 0
assert won_and_lost_amount_constants['n_buy_lost'] == 0
assert won_and_lost_amount_constants['n_total_days_in_trades'] == 0
won_and_lost_amount_constants

{'dollar_amount_buy_lost': 0.0,
 'dollar_amount_buy_won': 0.0,
 'dollar_amount_short_lost': 0.0,
 'dollar_amount_short_won': 0.0,
 'n_buy_lost': 0.0,
 'n_buy_won': 0.0,
 'n_short_lost': 0.0,
 'n_short_won': 0.0,
 'n_total_days_in_trades': 0.0}

In [28]:
bitcoin_df = read_in_data("../tmp/" + constants["bitcoin_csv_filename"].split("/")[1], running_on_aws())
etherum_df = read_in_data("../tmp/" +constants["etherum_csv_filename"].split("/")[1], running_on_aws())
sol_df = read_in_data("../tmp/" +constants["sol_csv_filename"].split("/")[1], running_on_aws())
matic_df = read_in_data("../tmp/" +constants["matic_csv_filename"].split("/")[1], running_on_aws())
link_df = read_in_data("../tmp/" +constants["link_csv_filename"].split("/")[1], running_on_aws())
tbt_df = read_in_data("../tmp/" +constants["tbt_csv_filename"].split("/")[1], running_on_aws())

In [29]:
ml_constants

{'prediction_params': {'bollinger_window': 5,
  'no_of_std': 1.25,
  'lookback_window': [15, 30, 45],
  'prediction_n_days': 7,
  'model_name': ['TCN', 'NBEATS'],
  'work_dir': '/mnt/shared-storage'},
 'hyperparameters_tcn': {'dropout': 0.1,
  'random_state': 0,
  'dilation_base': 2,
  'kernel_size': 14,
  'num_filters': 7,
  'num_layers': 7,
  'weight_norm': True,
  'epochs': 30},
 'hyperparameters_nbeats': {'random_state': 0,
  'num_blocks': 4,
  'layer_widths': 123,
  'epochs': 5}}

In [30]:
tbt_df

,open,high,low,close,volume
date,,,,,
2017-04-06,38.64,39.08,38.52,38.64,2231918
2017-04-07,38.26,38.96,38.02,38.95,3527348
2017-04-08,38.26,38.96,38.02,38.95,3527348
2017-04-09,38.26,38.96,38.02,38.95,3527348
2017-04-10,38.68,38.79,38.41,38.62,1662011
...,...,...,...,...,...
2022-04-01,21.08,21.21,20.22,20.42,6913069
2022-04-02,21.08,21.21,20.22,20.42,6913069
2022-04-03,21.08,21.21,20.22,20.42,6913069


In [31]:
bitcoin_df

,open,high,low,close,volume
date,,,,,
2017-01-01,963.66,1003.08,958.70,998.33,1.477750e+08
2017-01-02,998.62,1031.39,996.70,1021.75,2.221850e+08
2017-01-03,1021.60,1044.08,1021.60,1043.84,1.851680e+08
2017-01-04,1044.40,1159.42,1044.40,1154.73,3.449460e+08
2017-01-05,1156.73,1191.10,910.42,1013.38,5.101990e+08
...,...,...,...,...,...
2022-04-01,45518.00,46710.00,44180.00,46300.00,9.645929e+07
2022-04-02,46300.00,47203.00,45644.00,45817.00,5.006270e+07
2022-04-03,45817.00,47445.00,45556.00,46405.00,3.790648e+07


 - BTC

## SEtup the two main classes
- predict prices
- update trading state (bollinger bands)

In [32]:
# ml_constants['hyperparameters_tcn']['epochs']=1
# ml_constants['hyperparameters_nbeats']['epochs']=1

In [33]:
ml_constants

{'prediction_params': {'bollinger_window': 5,
  'no_of_std': 1.25,
  'lookback_window': [15, 30, 45],
  'prediction_n_days': 7,
  'model_name': ['TCN', 'NBEATS'],
  'work_dir': '/mnt/shared-storage'},
 'hyperparameters_tcn': {'dropout': 0.1,
  'random_state': 0,
  'dilation_base': 2,
  'kernel_size': 14,
  'num_filters': 7,
  'num_layers': 7,
  'weight_norm': True,
  'epochs': 30},
 'hyperparameters_nbeats': {'random_state': 0,
  'num_blocks': 4,
  'layer_widths': 123,
  'epochs': 5}}

### Write a loop going through each day in our main df
- Filter so that the largest lookback window is satisfied
- run price prediction and determine trading state
- go on to next date
- ..etc

In [34]:
max_lookback = np.max(ml_constants['prediction_params']['lookback_window']) # days

In [35]:
max_lookback

45

- Setups our DFs to run training against

In [36]:
date_slice = '2019-01-01'

In [37]:
choosen_df = bitcoin_df[bitcoin_df.index >= date_slice]
correl_df_1 =  tbt_df[tbt_df.index >= date_slice]
correl_df_2 =  etherum_df[etherum_df.index >= date_slice]

In [38]:
choosen_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1191 entries, 2019-01-01 to 2022-04-05
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1191 non-null   float64
 1   high    1191 non-null   float64
 2   low     1191 non-null   float64
 3   close   1191 non-null   float64
 4   volume  1191 non-null   float64
dtypes: float64(5)
memory usage: 55.8 KB


In [39]:
correl_df_1.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1191 entries, 2019-01-01 to 2022-04-05
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    1191 non-null   float64
 1   high    1191 non-null   float64
 2   low     1191 non-null   float64
 3   close   1191 non-null   float64
 4   volume  1191 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 55.8 KB


In [40]:
COIN_TO_PREDICT = "btc"

for  day_idx, day in enumerate(choosen_df.index):
    

    if (day_idx ) > max_lookback*2:
        print(f" Pct complete = {day_idx/len(bitcoin_df):.2f}")
        new_choosen_df = choosen_df[choosen_df.index <= day]
        new_correl_df1 = correl_df_1[correl_df_1.index <= day]
        new_correl_df2 = correl_df_2[correl_df_2.index <= day]
        # read in the new config files in case the yhave been updated
        coin, constants , trading_constants, won_and_lost_amount_constants,\
        actions_to_take_constants, ml_constants, all_predictions_filename, trading_state_filename, actions_to_take_filename, \
        won_lost_amount_filename= read_in_config_files(coin=COIN_TO_PREDICT)
        
        predictor = CoinPricePredictor(
            coin, constants, ml_constants, new_choosen_df,
            all_predictions_filename=all_predictions_filename, 
            additional_dfs=[new_correl_df1, new_correl_df2], verbose=False  # spy_df
        )

        price_prediction = predictor.predict()

        trading_state_class = DetermineTradingState(
            coin,
            price_prediction,
            constants,
            trading_constants,
            predictor.df,
            won_and_lost_amount_constants,
            actions_to_take_constants,
            running_on_aws(),
            # is_training=True
        )
        # TODO: read in yaml files again
        sys.stdout.flush()
        trading_state_class.calculate_positions()
        print("---- Finished determinig trading strategy --- ")
        trading_state_class.update_state()
        # this works
        if trading_state_class.actions_to_take_constants['action_to_take'] != 'none_to_none':
            print(trading_state_class.trading_state_constants, 'trading_state_class.trading_state_constants')
            print(trading_state_class.actions_to_take_constants, 'trading_state_class.actions_to_take_constants')
            print(trading_state_class.won_and_lose_amount_dict, 'trading_state_class.won_and_lose_amount_dict')

        update_yaml_config(trading_state_filename, trading_state_class.trading_state_constants, running_on_aws())
        update_yaml_config(won_lost_amount_filename, trading_state_class.won_and_lose_amount_dict, running_on_aws())
        update_yaml_config(actions_to_take_filename, trading_state_class.actions_to_take_constants, running_on_aws())
        
        

 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig trading strategy --- 
 Pct complete = 0.05
---- Finished determinig tr

KeyboardInterrupt: 

In [ ]:
new_eth_df

In [42]:
new_bitcoin_df

,open,high,low,close,volume
date,,,,,
2017-01-01,963.66,1003.08,958.70,998.33,147775008.0
2017-01-02,998.62,1031.39,996.70,1021.75,222184992.0
2017-01-03,1021.60,1044.08,1021.60,1043.84,185168000.0
2017-01-04,1044.40,1159.42,1044.40,1154.73,344945984.0
2017-01-05,1156.73,1191.10,910.42,1013.38,510199008.0
...,...,...,...,...,...
2017-03-29,1046.08,1055.13,1015.88,1039.97,298457984.0
2017-03-30,1042.21,1049.29,1020.04,1026.43,352968992.0
2017-03-31,1026.64,1074.92,1026.64,1071.79,447287008.0


In [41]:
df

NameError: name 'df' is not defined

2022-04-01 15:35:49,645: : __init__  Setting the class var self.buy_entry_price = 45520.0
2022-04-01 15:35:49,649: : __init__  Setting the class var self.buy_has_crossed_mean = 0.0
2022-04-01 15:35:49,650: : __init__  Setting the class var self.mode = buy
2022-04-01 15:35:49,651: : __init__  Setting the class var self.position_entry_date = 2022-03-31
2022-04-01 15:35:49,653: : __init__  Setting the class var self.short_entry_price = 0.0
2022-04-01 15:35:49,655: : __init__  Setting the class var self.short_has_crossed_mean = 0.0
2022-04-01 15:35:49,656: : __init__  Setting the class var self.stop_loss_pct = 0.1
2022-04-01 15:35:49,660: : __init__  Setting the class var self.stop_loss_price = 40968.0
2022-04-01 15:35:49,663: : __init__  Setting the class var self.dollar_amount_buy_lost = 0.0
2022-04-01 15:35:49,665: : __init__  Setting the class var self.dollar_amount_buy_won = 2813.0
2022-04-01 15:35:49,672: : __init__  Setting the class var self.dollar_amount_short_lost = 0.0
2022-04-0

### Update configs

2022-04-01 09:51:07,611: : calculate_positions  current row =                open     high      low    close        volume  Rolling Mean  \
date                                                                         
2022-03-31  47085.0  47680.0  45133.0  45520.0  7.928625e+07       46812.6   

            Bollinger High  Bollinger Low    stc      stoch        rsi  \
date                                                                     
2022-03-31    47756.439234   45868.760766  100.0  66.480447  51.345275   

                   macd  macd_signal  
date                                  
2022-03-31  1700.536619  1255.298319  
2022-04-01 09:51:07,622: : calculate_positions  prev_row =                open     high      low    close        volume  Rolling Mean  \
date                                                                         
2022-03-30  47455.0  47716.0  46329.0  47085.0  5.690887e+07       46614.8   

            Bollinger High  Bollinger Low    stc      stoch        rs